In [49]:
import pandas as pd
import numpy as np
import json

### 전국관광지표준데이터 사용

In [50]:
df = pd.read_csv('data.csv', encoding='cp949') # 공공데이터는 주로 cp949로 인코딩 되어있음
# null값 처리
df = df.replace({np.nan: None})
df = df.replace({'  ': " "})
df.head()

,관광지명,관광지구분,소재지도로명주소,소재지지번주소,위도,경도,면적,공공편익시설정보,숙박시설정보,운동및오락시설정보,...,지원시설정보,지정일자,수용인원수,주차가능수,관광지소개,관리기관전화번호,관리기관명,데이터기준일자,제공기관코드,제공기관명
0,일제 경성호국신사 계단(108계단),관광지,서울특별시 용산구 신흥로22가길 33,서울특별시 용산구 용산동2가 1-346,37.546023,126.982542,237.2,승강기,None,None,...,None,1905-04-26,20,0,"일제강점기 경성호국신사에 참배하기 위해 오르던 진입로로, 완공까지 한국인들의 재산과...",02-2199-7240,서울특별시 용산구청 문화체육과,2022-12-07,3020000,서울특별시 용산구
1,찬바람재,관광지,서울특별시 용산구 녹사평대로 195,서울특별시 용산구 용산동2가 7-99,37.534815,126.987014,646.6,승강기,None,None,...,None,2000-12-15,50,0,예전부터 겨울철 찬바람이 매섭게 몰아친다 하여 찬바람재 또는 한풍재라고도 불리던 곳...,02-2199-7240,서울특별시 용산구청 문화체육과,2022-12-07,3020000,서울특별시 용산구
2,옛 남영동 대공분실(민주인권기념관),관광지,서울특별시 용산구 한강대로71길 37,서울특별시 용산구 갈월동 98-8,37.541012,126.971634,6391.0,화장실,None,None,...,None,2024-12-01,430,0,1970~80년대 민주주의를 억압하며 악명을 떨쳤던 남영동 대공분실은 1976년 지...,031-361-9576,민주화운동기념사업회,2022-12-07,3020000,서울특별시 용산구
3,옛 용산공설시장(현 남영아케이드),관광지,서울특별시 용산구 한강대로84길 7,서울특별시 용산구 남영동 28-8,37.543723,126.972924,1449.0,화장실,None,None,...,None,1920-01-01,100,0,1922년 일제가 설치한 시장으로 지금은 남영아케이드로 불리고 있다.,02-2199-7240,서울특별시 용산구청 문화체육과,2022-12-07,3020000,서울특별시 용산구
4,전쟁기념관,관광지,서울특별시 용산구 이태원로 29,서울특별시 용산구 용산동1가 8,37.537314,126.978466,112887.2,화장실+주차장+물품보관함+수유실,None,None,...,None,1994-06-10,7500,150,1994년 용산에 건립된 전쟁기념관은 외침을 극복하고 국민의 생명과 재산을 지켜온 ...,02-709-3114,전쟁기념사업회,2022-12-07,3020000,서울특별시 용산구


### 그래프 데이터 생성

- Nodes와 Linkes로 구성된 그래프 데이터를 생성해야함
- Nodes는 id값과 관광지에 대한 정보를 담고 있음
- Links는 source와 target으로 구성되어 있으며, source와 target은 Nodes의 id값을 참조함
    - 이 데이터는 제공기관 기준으로 관광지의 연결관계를 나타내도록 정제함

In [51]:
# 관광지 식별을 위한 id 컬럼 추가
df['id'] = "tour" + df.index.astype(str)
# 관광지 id를 index로 설정
df.set_index('id', inplace=True)

In [52]:
# node와 link 관계 생성을 위해 dict 형태로 데이터 가공

df_dict = df.to_dict(orient='index')

In [53]:
graph_json = {"nodes": [], "links": []}

for index, item in df_dict.items():
    # 모든 컬럼 정보를 node의 속성으로 추가
    node = {"id": str(index)}
    for column, value in item.items():
        node[column] = value
    graph_json["nodes"].append(node)

    # link 관계를 위한 target 설정
    link_targets = [
        item['제공기관코드']
    ]

    for target in link_targets:
        if target is not None:
            graph_json["links"].append({"source": str(index), "target": str(target)})
            # target도 node로 추가
            # target이 이미 추가되었는지 확인
            if not any(d['id'] == str(target) for d in graph_json["nodes"]):
                graph_json["nodes"].append({"id": str(target), "name": item['제공기관명']})

In [54]:
#graph_json 노드의 0번째 데이터 확인
graph_json["nodes"][0]

{'id': 'tour0',
 '관광지명': '일제 경성호국신사 계단(108계단)',
 '관광지구분': '관광지',
 '소재지도로명주소': '서울특별시 용산구 신흥로22가길 33',
 '소재지지번주소': '서울특별시 용산구 용산동2가 1-346',
 '위도': 37.546023,
 '경도': 126.982542,
 '면적': 237.2,
 '공공편익시설정보': '승강기',
 '숙박시설정보': None,
 '운동및오락시설정보': None,
 '휴양및문화시설정보': None,
 '접객시설정보': None,
 '지원시설정보': None,
 '지정일자': '1905-04-26',
 '수용인원수': 20,
 '주차가능수': 0,
 '관광지소개': '일제강점기 경성호국신사에 참배하기 위해 오르던 진입로로, 완공까지 한국인들의 재산과 노동력이 동원되었다. 일제 말기 식민지 한국이 처했던 비극의 역사를 선명하게 보여주는 장소이다.',
 '관리기관전화번호': '02-2199-7240',
 '관리기관명': '서울특별시 용산구청 문화체육과',
 '데이터기준일자': '2022-12-07',
 '제공기관코드': 3020000,
 '제공기관명': '서울특별시 용산구'}

In [55]:
#graph_json 링크의 0번째 데이터 확인
graph_json["links"][0]

{'source': 'tour0', 'target': '3020000'}

In [56]:
print("nodes의 총 개수: ", len(graph_json["nodes"]))
print("links의 총 개수: ", len(graph_json["links"]))

nodes의 총 개수:  1054
links의 총 개수:  879


In [57]:
# graph_json를 json 파일로 저장
with open(f'data.json', 'w') as outfile:
    json.dump(graph_json, outfile, ensure_ascii=False, indent=4)